In [ ]:
import openai
import re
from typing import Literal
import pickle
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm
from openai import AsyncOpenAI, OpenAI
import random

In [ ]:
# Fill in name of finetuned_model.
MODEL_NAME = ""

DATASET: Literal["beauty", "steam"] = "beauty"

# Name of the pickle with the test data for Beauty.
TEST_DATA_PICKLE_NAME = f"test_data_{DATASET}.pickle"

# Name of the embeddings DF for 
EMBEDDINGS_NAME = f"embeddings_{DATASET}.csv.gz"

# Points to the pickle with the recommendation to rerank
OTHER_MODEL_RECOMMENDATIONS = f"llmseqsim_{DATASET}_recommendations.pkl"

# Fill in OpenAI key
OPENAI_KEY = ""

# Hyperparameters
TOP_K = 20
TEMPERATURE = 0
TOP_P = 1.0

# Correspond to respectively 4.1 to 4.4
VARIANT: Literal["genitem", "genlist", "classify", "rank"] = "rank"

TOTAL_MODEL_NAME = f"{MODEL_NAME}_{VARIANT}_temp_{TEMPERATURE}_top_p_{TOP_P}"

In [ ]:
def parse_completion_rank(completion: str) -> list[str]:
    recommendations = []
    items = re.findall(r"\d+\.\s(.*?)\n", completion)
    for i, item in enumerate(items, start=1):
        recommendations.append(item)
    return recommendations[:TOP_K]

In [ ]:
system_message = {
    "role": "system",
    "content": "You are a recommender system assistant.\nYou have access to the user's previous purchases and a list of availabe products.\n Provide 20 product recommendations for this user, only select from the available products.\n",
}
user_message = {
    "role": "user",
    "content": """
The user's previous purchases: 
{user_item_list}

Available products:
{recommendations_to_rerank}


Please remember to only select recommendations from the available products.
""",
}
parse_method = parse_completion_rank

## Load test prompts

In [ ]:
test_prompts, _ = pickle.load(open(f"{TEST_DATA_PICKLE_NAME}", "rb"))
test_prompts[list(test_prompts.keys())[0]]

In [ ]:
test_recommendations_from_seqsim = pickle.load(open(OTHER_MODEL_RECOMMENDATIONS, "rb"))
test_recommendations_from_seqsim[list(test_prompts.keys())[0]]

## Get embeddings and build lookup tables

In [ ]:
product_embeddings = pd.read_csv(
    f"{EMBEDDINGS_NAME}", compression="gzip"
)
product_embeddings

In [ ]:
product_id_to_name = (
    product_embeddings[["ItemId", "name"]]
    .set_index("ItemId")
    .to_dict()["name"]
)
product_name_to_id = (
    product_embeddings[["ItemId", "name"]]
    .set_index("name")
    .to_dict()["ItemId"]
)
product_index_to_embedding = (
    product_embeddings[["ItemId", "embedding"]]
    .set_index("ItemId")
    .to_dict()["embedding"]
)
product_index_to_embedding = {
    k: np.array(json.loads(v)) for k, v in product_index_to_embedding.items()
}
product_index_to_embedding = np.array(list(product_index_to_embedding.values()))
product_index_to_id = list(product_id_to_name.keys())
product_id_to_index = {idx: i for i, idx in enumerate(product_index_to_id)}

## Compute test prompts

In [ ]:
test_messages: list[tuple[int, list[str]]] = []

for session_id, prompt in test_prompts.items():
    custom_user_message = user_message.copy()
    custom_user_message["content"] = custom_user_message["content"].replace("{user_item_list}", "\n".join([product_id_to_name[i] for i in prompt]))
    
    seq_sim_recs = [product_id_to_name[i] for i in test_recommendations_from_seqsim[session_id]]
    random.shuffle(seq_sim_recs)
    custom_user_message["content"] = custom_user_message["content"].replace("{recommendations_to_rerank}", "\n".join(seq_sim_recs))
    test_messages.append((session_id, [system_message, custom_user_message]))
test_messages[0]

# Compute completions

In [ ]:
import asyncio
import time
completions: list[tuple[int, str]] = []

# Use async API to get parallel requests.
# Make sure batch_size is not too high otherwise we might hit rate limits.
async def run_completions():
    client = AsyncOpenAI(
        api_key=OPENAI_KEY,
    )

    batch_size = 150
    for i in tqdm(range(0, len(test_messages), batch_size)):
        start_batch = i
        end_batch = i + batch_size

        start_time = time.perf_counter()
        print(f"Completion batch {start_batch} - {end_batch}")

        requests = []
        for _, messages in test_messages[start_batch:end_batch]:
            requests.append(
                client.chat.completions.create(
                    model=MODEL_NAME,
                    temperature=TEMPERATURE,
                    top_p=TOP_P,
                    messages=messages,
                )
            )
        responses = await asyncio.gather(*requests)
        for (session_id, _), response in zip(test_messages[start_batch:end_batch], responses):
            completions.append((session_id, response.choices[0].message.content))
            
        print(f"Finished batch {start_batch} - {end_batch}. Took {time.perf_counter() - start_time} seconds.")


await run_completions()

In [ ]:
pickle.dump(completions, open(f"completions_openai_{total_model_name}", "wb"))

### Parse completions


In [ ]:
parsed_completions: list[tuple[int, list[str]]] = []
for session_id, response in tqdm(completions):
    parsed_response: list[str] = parse_method(response)
    if len(parsed_response) > TOP_K:
    if parsed_response is None:
        break
    parsed_completions.append((session_id, parsed_response))
parsed_completions[0]

# Completed product names to global product ids
First we try to map to the exact product name and otherwise we use embeddings to find the closest item. 

In [ ]:
from thefuzz import process
recommendations: dict[int, list[int]] = {}
unmappable_items: set = set()
duplicate_count = 0
exact_match = 0
not_exact_match = 0
for session_id, items in tqdm(parsed_completions):
    recommendations[session_id] = []

    seq_sim_recs = [product_id_to_name[i] for i in test_recommendations_from_seqsim[session_id]]
    seq_sim_recs_set = set(seq_sim_recs)

    items_so_far = set()

    # Map to item from rerank list
    for item in items:
        if item in seq_sim_recs_set:
            exact_match += 1
            match_from_seq_sim = item
        else:
            not_exact_match += 1
            match_from_seq_sim, match_score = process.extractOne(item, seq_sim_recs)
            
        if match_from_seq_sim in items_so_far:
            # Fall back on the next closest
            for match, match_score in process.extract(item, seq_sim_recs, limit=len(seq_sim_recs)):
                if match not in items_so_far:
                    match_from_seq_sim = match
            duplicate_count += 1
        
        recommendations[session_id].append(product_name_to_id[match_from_seq_sim])
        items_so_far.add(match_from_seq_sim)

    if len(items_so_far.intersection(seq_sim_recs_set)) != len(items_so_far):
        print("Panic")

print(duplicate_count)
print(exact_match)
print(not_exact_match)

len(recommendations), recommendations[list(recommendations.keys())[0]]   

Find closest actual item (with global product id) . Try to prevent duplicates.

# Save file

In [ ]:
pickle.dump(recommendations, open(f"recs_openai_{total_model_name}", "wb"))